<a href="https://colab.research.google.com/github/DigdinLogistico/API-1-FATEC/blob/main/Tratamento_de_dados_API_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
caminho = '/content/drive/My Drive/DATA/COMEXSTAT/'

In [ ]:
#Dados de exportação
exp_2023 = pd.read_csv(caminho + 'EXP_2023.csv', sep= ';',encoding='latin1')
exp_2024 = pd.read_csv(caminho + 'EXP_2024.csv', sep=';', encoding='latin-1')
exp_2025 = pd.read_csv(caminho + 'EXP_2025.csv', sep=';', encoding='latin-1')
exp_2023_mun = pd.read_csv(caminho + 'EXP_2023_MUN.csv', sep=';', encoding='latin-1')
exp_2024_mun = pd.read_csv(caminho + 'EXP_2024_MUN.csv', sep=';', encoding='latin-1')
exp_2025_mun = pd.read_csv(caminho + 'EXP_2025_MUN.csv', sep=';', encoding='latin-1')

#Dados de importação
imp_2023 = pd.read_csv(caminho + 'IMP_2023.csv', sep= ';',encoding='latin1')
imp_2024 = pd.read_csv(caminho + 'IMP_2024.csv', sep=';', encoding='latin-1')
imp_2025 = pd.read_csv(caminho + 'IMP_2025.csv', sep=';', encoding='latin-1')
imp_2023_mun = pd.read_csv(caminho + 'IMP_2023_MUN.csv', sep=';', encoding='latin-1')
imp_2024_mun = pd.read_csv(caminho + 'IMP_2024_MUN.csv', sep=';', encoding='latin-1')
imp_2025_mun = pd.read_csv(caminho + 'IMP_2025_MUN.csv', sep=';', encoding='latin-1')

#Colunas auxiliares
ncm  = pd.read_csv(caminho + "NCM.csv", sep=";", encoding="latin1")
pais = pd.read_csv(caminho + "PAIS.csv", sep=";", encoding="latin1")
urf  = pd.read_csv(caminho + "URF.csv", sep=";", encoding="latin1")
via  = pd.read_csv(caminho + "VIA.csv", sep=";", encoding="latin1")
ncm_uni = pd.read_csv(caminho + "NCM_UNIDADE.csv", sep=";", encoding="latin1")
uf_mun = pd.read_csv(caminho + "UF_MUN.csv", sep=";", encoding="latin1")
uf = pd.read_csv(caminho + "UF.csv", sep=";", encoding="latin1")

In [ ]:
#Compilar as tabelas em uma só
df_final = pd.concat([exp_2023, exp_2024, exp_2025, imp_2023, imp_2024, imp_2025, exp_2023_mun, exp_2024_mun, exp_2025_mun, imp_2023_mun, imp_2024_mun, imp_2025_mun], ignore_index=True)

In [ ]:
#Consolidar colunas de códigos e nomes
df_final = df_final.merge(ncm[["CO_NCM", "NO_NCM_POR"]], on="CO_NCM", how="left")
df_final = df_final.merge(pais[["CO_PAIS", "NO_PAIS"]], on="CO_PAIS", how="left")
df_final = df_final.merge(urf[["CO_URF", "NO_URF"]], on="CO_URF", how="left")
df_final = df_final.merge(via[["CO_VIA", "NO_VIA"]], on="CO_VIA", how="left")
df_final = df_final.merge(uf_mun[["CO_MUN_GEO", "NO_MUN"]], left_on="CO_MUN", right_on="CO_MUN_GEO", how="left")
# Merge the 'uf' DataFrame using 'SG_UF_NCM' and 'SG_UF', also consider 'SG_UF_MUN'
df_final = df_final.merge(uf[["SG_UF", "NO_UF"]], left_on=df_final['SG_UF_NCM'].fillna(df_final['SG_UF_MUN']), right_on="SG_UF", how="left")

In [ ]:
print(df_final.columns)

Index(['CO_ANO', 'CO_MES', 'CO_NCM', 'CO_UNID', 'CO_PAIS', 'SG_UF_NCM',
       'CO_VIA', 'CO_URF', 'QT_ESTAT', 'KG_LIQUIDO', 'VL_FOB', 'VL_FRETE',
       'VL_SEGURO', 'SH4', 'SG_UF_MUN', 'CO_MUN', 'NO_NCM_POR', 'NO_PAIS',
       'NO_URF', 'NO_VIA', 'CO_MUN_GEO', 'NO_MUN', 'SG_UF', 'NO_UF'],
      dtype='object')


In [ ]:
#Renomear colunas de dados

colunas_interesse = {
    "CO_ANO": "Ano",
    "CO_MES": "Mes",
    "NO_UF": "Estado",
    "CO_VIA": "Modal",
    "QT_ESTAT": "Quantidade",
    "NO_NCM_POR": "Produto",
    "KG_LIQUIDO": "Peso_Liquido",
    "NO_PAIS": "Pais_Destino",
    "VL_FOB": "Valor_Carga",
    "CO_MUN_GEO": "Município"
}

df_final = df_final[list(colunas_interesse.keys())].rename(columns=colunas_interesse)

In [ ]:
print(df_final.columns)

Index(['Ano', 'Mes', 'Estado', 'Modal', 'Quantidade', 'Produto',
       'Peso_Liquido', 'Pais_Destino', 'Valor_Carga', 'Município'],
      dtype='object')


In [ ]:
# Textos
df_final["Estado"] = df_final["Estado"].astype(str).str.strip().str.upper()
df_final["Município"] = df_final["Município"].astype(str).str.strip().str.title()
df_final["Produto"] = df_final["Produto"].astype(str).str.strip()
df_final["Pais_Destino"] = df_final["Pais_Destino"].astype(str).str.strip().str.title()

# Números
df_final["Quantidade"] = pd.to_numeric(df_final["Quantidade"], errors="coerce").fillna(0).astype(int)
df_final["Valor_Carga"] = pd.to_numeric(df_final["Valor_Carga"], errors="coerce").fillna(0)

In [ ]:
#Salvar o arquivo tratado

df_final.to_csv(caminho + "Importação_Exportação_1.csv", sep=";", index=False, encoding="latin1")
print("✅ Arquivo consolidado gerado com sucesso!")

#Arquivo com dados básicos.

✅ Arquivo consolidado gerado com sucesso!


In [ ]:
# Carregar o arquivo unificado pesado
df = pd.read_csv(caminho + "Importação_Exportação_1.csv", sep=';', encoding="latin1")

# ===== 1. Remover colunas duplicadas =====
df = df.loc[:, ~df.columns.duplicated()]

# ===== 2. Remover linhas duplicadas =====
df = df.drop_duplicates()

# ===== 3. Remover colunas totalmente vazias =====
df = df.dropna(axis=1, how="all")

# ===== 4. Remover espaços extras dos nomes das colunas =====
df.columns = df.columns.str.strip()

# ===== 5. Remover colunas pouco úteis =====
for col in df.columns:
    if df[col].nunique() <= 1:
        df = df.drop(columns=col)

# ===== 6. Salvar o arquivo reduzido =====
df.to_csv(caminho + "Importação_exportação_2.csv", sep=";", index=False, encoding="latin1")

print("Arquivo final salvo! Tamanho:", df.shape)

/tmp/ipython-input-628938606.py:2: DtypeWarning: Columns (5,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(caminho + "Importação_Exportação_1.csv", sep=';', encoding="latin1")


Arquivo final salvo! Tamanho: (17787197, 10)
